In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated \ufeffApril 17, 2020\r\n']

In [5]:
m = re.search(".*pdated (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
data_date_str = m.group(1)
data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
data_date

'2020-04-17'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                   8930   
1                            7,400 (83%)   
2                            1,530 (17%)   
3        Among those with race specified   
4                              23 (1.5%)   
5                                59 (4%)   
6                            437 (28.5%)   
7                               7 (0.5%)   
8                            989 (64.5%)   
9                              15 (1.0%)   
10                                   NaN   
11                           7,387 (83%)   
12                           1,543 (17%)   
13  Among those with ethnicity specified   
14                             500 (32%)   
15                           1,043 (68%)   

                                          \
                                   18-44   
0                                 179938   
1                          144,315 (80%)   
2                           35,623 (20%)   
3        Among those with race specified   
4                               310 (1%)   
5                             1,823 (5%)   
6                           11,018 (31%)   
7                             160 (0.5%)   
8                           22,170 (62%)   
9                             142 (0.5%)   
10                                   NaN   
11                         145,083 (81%)   
12                          34,855 (19%)   
13  Among those with ethnicity specified   
14                           8,637 (25%)   
15                          26,218 (75%)   

                                          \
                                   45-64   
0                                 183722   
1                          141,222 (77%)   
2                           42,500 (23%)   
3        Among those with race specified   
4                             277 (0.7%)   
5                             2,003 (5%)   
6                           14,218 (33%)   
7                             136 (0.3%)   
8                         25,748 (60.7%)   
9                             118 (0.3%)   
10                                   NaN   
11                         144,184 (78%)   
12                          39,538 (22%)   
13  Among those with ethnicity specified   
14                           7,508 (19%)   
15                          32,030 (81%)   

                                          \
                                   65-74   
0                                  53648   
1                           37,973 (71%)   
2                           15,675 (29%)   
3        Among those with race specified   
4                              87 (0.5%)   
5                               548 (3%)   
6                            5,293 (34%)   
7                              40 (0.3%)   
8                            9,674 (62%)   
9                              33 (0.2%)   
10                                   NaN   
11                          40,098 (75%)   
12                          13,550 (25%)   
13  Among those with ethnicity specified   
14                           1,603 (12%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)